In [ ]:
library(modeldata)
data(car_prices)
library(data.table)
library(tidyverse)
library(DBI)
library(RSQLite)
library(dbplyr)

In [ ]:
car_prices

In [ ]:
#Create two different tables with Mileage and Price as the common collumns
car_prices_1 <- car_prices %>% select(Mileage, Price, Chevy)
car_prices_2 <- car_prices %>% select(Mileage, Price, Cadillac)

In [ ]:
#Create sqlite connection, delete it if first if it exists
if (file.exists("~/databb/temp/ege_assignment_5.db")) file.remove("~/databb/temp/ege_assignment_5.db")
consq <- dbConnect(RSQLite::SQLite(), "~/databb/temp/ege_assignment_5.db")

In [ ]:
#Write the data tables into the sqlite database
dbWriteTable(consq, "car_prices_1", car_prices_1, overwrite = T)
dbWriteTable(consq, "car_prices_2", car_prices_2, overwrite = T)

In [ ]:
#Create lazy queries
car_prices_1_lazy = tbl(consq, "car_prices_1")
car_prices_2_lazy = tbl(consq, "car_prices_2")

In [ ]:
#Create lazy query object with select to just pick chevy and price collumns
query01 <- car_prices_1_lazy %>% select(Price, Chevy)

In [ ]:
#Working on the previous query, pick the ones that actually are chevy and reasonably priced(15000)
query02 <- query01 %>% filter(Chevy == 1 & Price < 15000)

In [ ]:
#Create a new Mileage/Price collumn to see how worth a car is
query03 <- car_prices_2_lazy %>% mutate(`Mileage/Price` = Mileage / Price)

In [ ]:
#Join by price to see how they compare if they have the same price
query04 <- query03 %>% left_join(query02)

In [ ]:
#Summarize different price points
query05 <- query04 %>% group_by(Price) %>% summarise(Chevy = Chevy, Cadillac = Cadillac, `Mileage/Price` = avg(`Mileage/Price`)) %>% ungroup

In [ ]:
query05 %>% show_query

In [ ]:
result = query05 %>% collect()

In [ ]:
result

In [ ]:
#Creating a datetime which is just the current time
current_time <- Sys.time()
current_time

In [ ]:
#Get the time part of the datetime
time = format(current_time , "%H:%M:%S")
time

In [ ]:
#Get the date part of the datetime
date = as.Date(current_time)
date

In [ ]:
#Get a periodic version of time to make it more human friendly
period_time <- time %>% hms
period_time

In [ ]:
#Find the first day of this month
floor_date(date, "month")